<a href="https://colab.research.google.com/github/KarynA19/Cloud-Computing/blob/main/Tirgul4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import ipywidgets as widgets
from IPython.display import display

# 1. טוען את הנתונים
url = 'https://data.gov.il/api/3/action/datastore_search?resource_id=053cea08-09bc-40ec-8f7a-156f0677aff3'
response = requests.get(url)
data = response.json()
#המרת המידע לצורה שך טבלה ולא של פסקה
data_df = pd.DataFrame(data['result']['records'])

# ממירים את שנת הייצור למספרים ממיר לInt
data_df['shnat_yitzur'] = pd.to_numeric(data_df['shnat_yitzur'], errors='coerce')

# טאב 1: סטטיסטיקות כלליות
tab1 = widgets.Output()
with tab1:
    print("סטטיסטיקות כלליות:")
    #מציג סטטיסטיקות על עמודות מספריות בטבלה
    display(data_df.describe())

# טאב 2: טבלה מלאה
tab2 = widgets.Output()
with tab2:
    print("כל הנתונים:")
    display(data_df)

# טאב 3: גרף שנת ייצור
tab3 = widgets.Output()
with tab3:
    print("גרף של מספר רכבים לפי שנת ייצור:")
    plt.figure(figsize=(10, 6))
    sns.histplot(data=data_df, x='shnat_yitzur', bins=30, kde=True)
    plt.title('Distribution of vehicles by year of manufacture')
    plt.xlabel('shnat_yitzur')
    plt.ylabel('Frequencies')
    plt.grid(True)
    plt.show()

# תיבת בחירה - יצרן
tozeret_dropdown = widgets.Dropdown(
    options=[''] + sorted(data_df['tozeret_nm'].dropna().unique().tolist()),
    description='tozeret:',
    style={'description_width': 'initial'}
)

# תיבת בחירה - דגם
kinuy_dropdown = widgets.Dropdown(
    options=[''],
    description='kinuy mishari:',
    style={'description_width': 'initial'}
)

# תיבת פלט
output_info = widgets.Output()

# עדכון רשימת דגמים לפי יצרן
def update_models(change):
    selected = change['new']
    if selected:
        models = data_df[data_df['tozeret_nm'] == selected]['kinuy_mishari'].dropna().unique()
        kinuy_dropdown.options = [''] + sorted(models)
    else:
        kinuy_dropdown.options = ['']

# עדכון מידע על דגם
def update_info(change=None):
    output_info.clear_output()
    selected_make = tozeret_dropdown.value
    selected_model = kinuy_dropdown.value
    if selected_make and selected_model:
        filtered = data_df[
            (data_df['tozeret_nm'] == selected_make) &
            (data_df['kinuy_mishari'] == selected_model)
        ]
        count = len(filtered)
        ramatim = filtered['ramat_gimur'].dropna().unique()
        with output_info:
            print(f"מספר רכבים: {count}")
            print("רמות גימור קיימות:", ", ".join(ramatim) if len(ramatim) > 0 else "אין מידע")

# קישור בין הבחירות
tozeret_dropdown.observe(update_models, names='value')
kinuy_dropdown.observe(update_info, names='value')

# יצירת טאבים
tabs = widgets.Tab(children=[tab1, tab2, tab3])
tabs.set_title(0, 'Tab 1: סטטיסטיקות')
tabs.set_title(1, 'Tab 2: טבלה')
tabs.set_title(2, 'Tab 3: גרף')

# הצגת הממשק
display(tabs)
display(widgets.VBox([tozeret_dropdown, kinuy_dropdown, output_info]))
